In [ ]:
axios = require('axios')
_ = require('underscore')
token = ''
project = '2239629'
pivotal = axios.create({
  baseURL: 'https://www.pivotaltracker.com/services/v5/',
  timeout: 4000,
  headers: {'X-TrackerToken': token}
})

In [ ]:
// pivotal.get('me')
//     .then((x) => console.log(x.data))
//     .catch((err) => console.log(err))

In [ ]:
// pivotal.get(`projects/${project}`)
//     .then((x) => console.log(x.data))
//     .catch((err) => console.log(err))

In [ ]:
// pivotal.get(`projects/${project}/labels`)
//     .then((x) => console.log(x.data))
//     .catch((err) => console.log(err))

In [ ]:
getStories = async () => {

    date_to = new Date()
    date_from = new Date()
    date_from.setDate(date_to.getDate() - 7)

    epics = await pivotal.get(`projects/${project}/epics`)
        .then(({data}) => _.map(data, (x) => x.label.name))

    stories = await pivotal.get(`projects/${project}/stories`, {
        params: {
            updated_after: date_from
        }
    }).then(({data}) => data)
    
    workedOn = _.chain(stories)
    .filter((x) => ['accepted', 'delivered'].includes(x.current_state))
    .map(({name, labels}) => {
        labels = _.map(labels, (x) => x.name)
        return { name, labels }
    })
    .value()
    
    inEpics = _.filter(workedOn, ({name, labels}) => _.intersection(labels, epics).length > 0)
    others = _.filter(workedOn, ({name, labels}) => _.intersection(labels, epics).length < 1)
    
    groupByEpics = {'others': []}
    inEpics.forEach(({name, labels}) => {
        labels.forEach((label) => {
            if (!(label in groupByEpics)) {
                groupByEpics[label] = []
            }      
            groupByEpics[label].push(name)
        })
    })
    
    others.forEach(({name, labels}) => {
        groupByEpics.others.push(name)
    })
    
    for (let [key, value] of Object.entries(groupByEpics)) {
        console.log(`${key}:`)
        value.forEach((x) => console.log(`    ${x}`))
    }
}

In [ ]:
getStories()